# 1.5 Building Agentic Applications

This notebook shows how to **combine OpenAI's features** to build production-ready AI agents.

**Key Topics:**
- Web Search tool (real-time information)
- File Search tool (RAG with documents)
- Multimodal capabilities (images, text)
- Advanced streaming patterns
- Background mode (long-running tasks)
- **Case Study: Research Agent**

**Why this matters:** Real agents combine multiple capabilities to solve complex problems.

<a target="_blank" href="https://colab.research.google.com/github/IT-HUSET/ai-agents-course-2025/blob/main/exercises/1.6-agentic-applications.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

## Setup

In [ ]:
%pip install openai~=1.60 python-dotenv~=1.0 pillow requests --upgrade --quiet

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
from openai import OpenAI
import json

_ = load_dotenv(find_dotenv())
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

print("✅ Setup complete")

---

## Part 1: Web Search Tool

**Web search** allows your agent to access real-time information from the internet.

**Use cases:**
- Current events and news
- Latest documentation or API changes
- Real-time data (stock prices, weather, etc.)
- Fact-checking and verification

### Basic Web Search

In [ ]:
# Simple web search query
response = client.responses.create(
    model="gpt-5",
    input="What are the latest developments in AI agents as of 2025?",
    tools=[{"type": "web_search"}]
)

print(response.output_text)

### Inspecting Search Results

The response includes metadata about the web searches performed.

In [ ]:
# Check how many searches were performed
print(f"Response ID: {response.id}")
print(f"Model used: {response.model}")
print(f"\nOutput items: {len(response.output)}")

# Examine output structure
for i, item in enumerate(response.output):
    print(f"\nItem {i}:")
    print(f"  Type: {item.type}")
    if hasattr(item, 'content'):
        preview = str(item.content)[:100] + "..." if len(str(item.content)) > 100 else str(item.content)
        print(f"  Content preview: {preview}")

### Combining Web Search with Specific Questions

In [ ]:
# Research question requiring current information
research_query = """Research and compare the latest versions of LangGraph and LangChain.

Please provide:
1. Current version numbers
2. Major features added in the last 6 months
3. Key differences between the two frameworks
4. Which one is recommended for building AI agents in 2025?
"""

response = client.responses.create(
    model="gpt-5",
    input=research_query,
    tools=[{"type": "web_search"}]
)

print(response.output_text)

### 🎯 Exercise 1: Web Search Agent

**Task:** Create a news aggregator agent that:
1. Searches for recent news on a given topic
2. Summarizes the top 3 stories
3. Formats the output as a brief newsletter

**Test with topic:** "GPT-5 release" or "AI regulations 2025"

In [ ]:
# YOUR CODE HERE

def create_news_summary(topic: str) -> str:
    """Generate a news summary on a given topic"""
    # TODO: Create a prompt that searches and summarizes
    prompt = f"""TODO: Write a prompt that:
    1. Searches for recent news on: {topic}
    2. Finds top 3 most relevant stories
    3. Formats as a newsletter with headlines, summaries, and sources
    """
    
    # response = client.responses.create(
    #     model="gpt-5",
    #     input=prompt,
    #     tools=[{"type": "web_search"}]
    # )
    # return response.output_text
    pass

# Test it
# print(create_news_summary("AI regulations 2025"))

---

## Part 2: File Search Tool

**File search** enables RAG (Retrieval-Augmented Generation) with your own documents.

**Use cases:**
- Q&A over internal documentation
- Customer support knowledge bases
- Legal document analysis
- Research paper summaries

### Upload Files for Search

In [ ]:
# Example: Upload a text file
# Note: You'll need to create a sample file first

sample_content = """# Company Policy: Remote Work

## Overview
Our company supports flexible remote work arrangements for all employees.

## Eligibility
- All full-time employees are eligible after 3 months
- Part-time employees may request approval from their manager

## Equipment
- Laptop and monitor provided by IT department
- Monthly stipend of $50 for internet expenses
- Optional: Standing desk and ergonomic chair (upon request)

## Requirements
- Available during core hours (10 AM - 3 PM local time)
- Respond to messages within 2 hours during business hours
- Attend weekly team meetings via video

## Security
- Use company VPN for all work activities
- Keep software up to date
- Never share login credentials
"""

# Write sample file
with open("remote_work_policy.txt", "w") as f:
    f.write(sample_content)

# Upload to OpenAI
file = client.files.create(
    file=open("remote_work_policy.txt", "rb"),
    purpose="assistants"  # Required for file search
)

print(f"File uploaded: {file.id}")
print(f"Filename: {file.filename}")

### Query Documents with File Search

In [ ]:
# Ask questions about the uploaded document
response = client.responses.create(
    model="gpt-5",
    input="What equipment is provided for remote workers?",
    tools=[{
        "type": "file_search",
        "file_search": {
            "file_ids": [file.id]
        }
    }]
)

print(response.output_text)

In [ ]:
# Multiple questions in a conversation
questions = [
    "Who is eligible for remote work?",
    "What are the core hours?",
    "What security measures are required?"
]

previous_id = None

for question in questions:
    response = client.responses.create(
        model="gpt-5",
        input=question,
        tools=[{
            "type": "file_search",
            "file_search": {
                "file_ids": [file.id]
            }
        }],
        previous_response_id=previous_id
    )
    
    print(f"\nQ: {question}")
    print(f"A: {response.output_text}")
    
    previous_id = response.id

### 🎯 Exercise 2: Document Q&A System

**Task:** Create a simple document Q&A system:
1. Create a text file with information about a topic (e.g., Python best practices)
2. Upload it using the Files API
3. Build a function that answers questions about the document
4. Test with at least 3 different questions

In [ ]:
# YOUR CODE HERE

# 1. Create your document content
your_document = """TODO: Write content about a topic of your choice"""

# 2. Upload the file
# TODO: Write file and upload

# 3. Create Q&A function
def ask_document(question: str, file_id: str) -> str:
    """Ask a question about the uploaded document"""
    # TODO: Implement
    pass

# 4. Test with questions
# questions = [
#     "Question 1...",
#     "Question 2...",
#     "Question 3..."
# ]
# for q in questions:
#     print(f"Q: {q}")
#     print(f"A: {ask_document(q, file_id)}\n")

---

## Part 3: Multimodal Capabilities

The Responses API supports **images, text, and audio** in the same conversation.

**Use cases:**
- Image analysis and description
- Visual question answering
- Diagram interpretation
- Screenshot analysis

### Analyzing Images

In [ ]:
# Using a publicly available image URL
image_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/3/3a/Cat03.jpg/1200px-Cat03.jpg"

response = client.responses.create(
    model="gpt-5",
    input=[
        {
            "type": "text",
            "text": "Describe this image in detail."
        },
        {
            "type": "image_url",
            "image_url": {"url": image_url}
        }
    ]
)

print(response.output_text)

### Combining Image Analysis with Web Search

In [ ]:
# Analyze image and search for related information
response = client.responses.create(
    model="gpt-5",
    input=[
        {
            "type": "text",
            "text": "What breed is this cat? Search for information about this breed's characteristics and care requirements."
        },
        {
            "type": "image_url",
            "image_url": {"url": image_url}
        }
    ],
    tools=[{"type": "web_search"}]
)

print(response.output_text)

### Multiple Images in One Request

In [ ]:
# Compare multiple images
image_1 = "https://upload.wikimedia.org/wikipedia/commons/3/3a/Cat03.jpg"
image_2 = "https://upload.wikimedia.org/wikipedia/commons/thumb/4/4d/Cat_November_2010-1a.jpg/1200px-Cat_November_2010-1a.jpg"

response = client.responses.create(
    model="gpt-5",
    input=[
        {
            "type": "text",
            "text": "Compare these two cat images. What are the differences in appearance?"
        },
        {
            "type": "image_url",
            "image_url": {"url": image_1}
        },
        {
            "type": "image_url",
            "image_url": {"url": image_2}
        }
    ]
)

print(response.output_text)

### 🎯 Exercise 3: Screenshot Analysis Agent

**Task:** Create an agent that analyzes UI screenshots:
1. Takes a screenshot URL as input
2. Identifies UI elements (buttons, forms, navigation)
3. Suggests UX improvements
4. Searches for current UI/UX best practices (bonus)

**Test with:** Any public screenshot of a web application

In [ ]:
# YOUR CODE HERE

def analyze_ui_screenshot(screenshot_url: str, include_research: bool = False) -> str:
    """Analyze a UI screenshot and provide feedback"""
    # TODO: Implement
    pass

# Test with a screenshot URL
# screenshot = "https://example.com/screenshot.png"
# analysis = analyze_ui_screenshot(screenshot, include_research=True)
# print(analysis)

---

## Part 4: Stateful Conversations

The Responses API **automatically manages conversation history** using `previous_response_id`.

**Benefits:**
- No need to manually track messages
- Automatic context management
- Easy to continue conversations
- Built-in context window optimization

### Multi-Turn Conversation

In [ ]:
# Start a conversation
response_1 = client.responses.create(
    model="gpt-5-mini",
    input="I'm building a web scraper in Python. What libraries should I use?"
)

print("Turn 1:")
print(response_1.output_text)
print("\n" + "="*80 + "\n")

In [ ]:
# Continue the conversation
response_2 = client.responses.create(
    model="gpt-5-mini",
    input="How do I handle rate limiting?",
    previous_response_id=response_1.id
)

print("Turn 2:")
print(response_2.output_text)
print("\n" + "="*80 + "\n")

In [ ]:
# Ask a follow-up that requires context
response_3 = client.responses.create(
    model="gpt-5-mini",
    input="Show me a code example",
    previous_response_id=response_2.id
)

print("Turn 3:")
print(response_3.output_text)

### Retrieving Full Conversation History

In [ ]:
# Fetch the entire conversation
conversation = client.responses.retrieve(response_id=response_3.id)

print("Full conversation context:")
print(f"Response ID: {conversation.id}")
print(f"Total output items: {len(conversation.output)}")
print(f"\nFinal output:\n{conversation.output_text}")

### Conversation Manager Helper Class

In [ ]:
class ConversationManager:
    """Helper class for managing stateful conversations"""
    
    def __init__(self, model: str = "gpt-5-mini"):
        self.model = model
        self.last_response_id = None
        self.history = []
    
    def send(self, message: str, **kwargs) -> str:
        """Send a message and get response"""
        response = client.responses.create(
            model=self.model,
            input=message,
            previous_response_id=self.last_response_id,
            **kwargs
        )
        
        self.last_response_id = response.id
        self.history.append({
            "user": message,
            "assistant": response.output_text,
            "response_id": response.id
        })
        
        return response.output_text
    
    def reset(self):
        """Reset conversation"""
        self.last_response_id = None
        self.history = []
    
    def get_history(self) -> list:
        """Get conversation history"""
        return self.history

# Test the manager
conv = ConversationManager()

print("Turn 1:")
print(conv.send("What's the capital of France?"))
print("\n" + "="*80 + "\n")

print("Turn 2:")
print(conv.send("What's the population?"))
print("\n" + "="*80 + "\n")

print("Turn 3:")
print(conv.send("Name 3 famous landmarks"))

### 🎯 Exercise 4: Tutorial Bot

**Task:** Build a conversational tutorial bot:
1. Teaches a topic step-by-step (e.g., Git basics)
2. Asks if the user understands before moving on
3. Provides examples when requested
4. Remembers what has been covered

Use the `ConversationManager` class above as a starting point.

In [ ]:
# YOUR CODE HERE

class TutorialBot:
    """Interactive tutorial bot with step-by-step teaching"""
    
    def __init__(self, topic: str):
        self.topic = topic
        self.conv = ConversationManager(model="gpt-5")
        # TODO: Initialize with instructions
    
    def start(self):
        """Start the tutorial"""
        # TODO: Implement
        pass
    
    def next_lesson(self):
        """Move to next lesson"""
        # TODO: Implement
        pass
    
    def ask_question(self, question: str) -> str:
        """Ask a question about current topic"""
        # TODO: Implement
        pass

# Test your tutorial bot
# bot = TutorialBot("Git version control basics")
# bot.start()

---

## Part 5: Streaming Responses

**Streaming** allows you to display responses as they're generated, improving user experience.

**Benefits:**
- Lower perceived latency
- Better UX for long responses
- Can cancel if needed
- Real-time feedback

### Basic Streaming

In [ ]:
# Stream a response
stream = client.responses.create(
    model="gpt-5-mini",
    input="Explain how neural networks work in 3 paragraphs",
    stream=True
)

print("Streaming response:")
for chunk in stream:
    if hasattr(chunk, 'delta') and hasattr(chunk.delta, 'content'):
        print(chunk.delta.content, end='', flush=True)

print("\n\n✅ Stream complete")

### Streaming with Web Search

In [ ]:
# Stream responses that include web searches
stream = client.responses.create(
    model="gpt-5",
    input="What are the latest news about OpenAI?",
    tools=[{"type": "web_search"}],
    stream=True
)

print("Streaming with web search:")
for chunk in stream:
    if hasattr(chunk, 'delta') and hasattr(chunk.delta, 'content'):
        print(chunk.delta.content, end='', flush=True)

print("\n\n✅ Stream complete")

### 🎯 Exercise 5: Streaming Chat Interface

**Task:** Build a simple streaming chat interface:
1. Accept user input
2. Stream the response with visual indication
3. Support multiple turns (use previous_response_id)
4. Display "thinking" indicator while searching (if using web search)

In [ ]:
# YOUR CODE HERE

import time

def streaming_chat():
    """Simple streaming chat interface"""
    # TODO: Implement
    pass

# Run the chat
# streaming_chat()

---

## Part 6: Instructions Parameter

The `instructions` parameter provides **system-level guidance** that persists across the entire conversation.

**Use cases:**
- Setting agent personality
- Defining output format
- Establishing constraints
- Role definition

### Using Instructions

In [ ]:
# Define persistent instructions
response = client.responses.create(
    model="gpt-5-mini",
    instructions="""You are a Python expert who:
    - Always provides working code examples
    - Follows PEP 8 style guidelines
    - Explains complex concepts simply
    - Includes type hints in all code
    """,
    input="How do I read a CSV file?"
)

print(response.output_text)

### Instructions Persist Across Turns

In [ ]:
# Instructions apply to entire conversation
response_1 = client.responses.create(
    model="gpt-5-mini",
    instructions="""You are a technical writer. Always:
    1. Use simple language
    2. Provide examples
    3. Format as markdown
    4. Keep responses under 100 words
    """,
    input="What is a REST API?"
)

print("Turn 1:")
print(response_1.output_text)
print("\n" + "="*80 + "\n")

# Follow-up inherits instructions
response_2 = client.responses.create(
    model="gpt-5-mini",
    input="What about GraphQL?",
    previous_response_id=response_1.id
)

print("Turn 2 (instructions still apply):")
print(response_2.output_text)

### 🎯 Exercise 6: Code Review Agent

**Task:** Build a code review agent with instructions:
1. Define clear review criteria in instructions
2. Accept code snippets as input
3. Provide structured feedback (bugs, style, improvements)
4. Support follow-up questions

**Test with various code snippets**

In [ ]:
# YOUR CODE HERE

REVIEW_INSTRUCTIONS = """TODO: Define code review instructions"""

def review_code(code: str, language: str = "python") -> str:
    """Review code and provide feedback"""
    # TODO: Implement
    pass

# Test code sample
sample_code = """
def get_user(id):
    user = database.query(f"SELECT * FROM users WHERE id = {id}")
    return user
"""

# print(review_code(sample_code))

---

## Part 7: Advanced Streaming Patterns

Streaming with the Responses API uses **semantic events** for fine-grained control.

### Stream Events

In [ ]:
# Stream with event handling
print("Streaming with events:")
print("=" * 50)

stream = client.responses.create(
    model="gpt-4o",
    input="Write a short poem about coding",
    stream=True
)

for event in stream:
    if event.type == "response.created":
        print("\n🎬 Response started")
    elif event.type == "response.output_text.delta":
        print(event.delta, end="", flush=True)
    elif event.type == "response.completed":
        print("\n\n✅ Response complete")
    elif event.type == "error":
        print(f"\n❌ Error: {event.error}")

### Streaming Tool Calls

In [ ]:
# Stream with web search
stream = client.responses.create(
    model="gpt-4o",
    input="What are the latest AI developments in 2025?",
    tools=[{"type": "web_search"}],
    stream=True
)

print("Streaming with tool calls:")
print("=" * 50)

for event in stream:
    if event.type == "response.web_search.searching":
        print("\n🔍 Searching the web...")
    elif event.type == "response.output_text.delta":
        print(event.delta, end="", flush=True)
    elif event.type == "response.completed":
        print("\n\n✅ Complete")

---

## Part 8: Background Mode

**Background mode** runs long tasks asynchronously, perfect for reasoning models.

### Basic Background Execution

In [ ]:
import time

# Start background task
response = client.responses.create(
    model="o1-mini",
    input="Solve this complex math problem step by step: ...",
    background=True
)

print(f"Started: {response.id}")
print(f"Status: {response.status}")

# Poll for completion
while response.status in {"queued", "in_progress"}:
    print(f"⏳ Status: {response.status}")
    time.sleep(2)
    response = client.responses.retrieve(response.id)

if response.status == "completed":
    print(f"\n✅ Complete!")
    print(response.output_text)

### Streaming a Background Response

In [ ]:
# Start background + stream
stream = client.responses.create(
    model="o1-mini",
    input="Write a detailed analysis of...",
    background=True,
    stream=True
)

cursor = None
for event in stream:
    if event.type == "response.output_text.delta":
        print(event.delta, end="", flush=True)
    cursor = event.sequence_number

# If connection drops, resume from cursor:
# resumed = client.responses.stream(response_id, starting_after=cursor)

### Cancelling Background Tasks

In [ ]:
# Start long task
response = client.responses.create(
    model="o3",
    input="Very long task...",
    background=True
)

print(f"Started: {response.id}")

# Cancel if needed
cancelled = client.responses.cancel(response.id)
print(f"Cancelled: {cancelled.status}")

---

## Part 9: Case Study - Research Agent

Let's build a complete research agent that combines:
- Web search for current information
- File search for stored knowledge
- Structured outputs for reliability
- Streaming for UX

### Step 1: Define Schemas

In [ ]:
from pydantic import BaseModel
from typing import List

class Source(BaseModel):
    type: str  # "web" or "document"
    title: str
    url: str = None
    relevance: float  # 0-1

class ResearchFindings(BaseModel):
    topic: str
    summary: str
    key_points: List[str]
    sources: List[Source]
    confidence: float  # 0-1

### Step 2: Implement Research Agent

In [ ]:
class ResearchAgent:
    """Complete research agent combining multiple tools"""
    
    def __init__(self, knowledge_files: List[str] = None):
        self.knowledge_files = knowledge_files or []
    
    def research(self, topic: str, stream: bool = False) -> ResearchFindings:
        """Conduct research on a topic"""
        
        # Build tools list
        tools = [{"type": "web_search"}]
        
        if self.knowledge_files:
            tools.append({
                "type": "file_search",
                "file_search": {"file_ids": self.knowledge_files}
            })
        
        # Create prompt
        prompt = f"""Research this topic thoroughly: {topic}
        
        Use web search for current information and documents for background.
        Synthesize findings into a comprehensive report.
        """
        
        if stream:
            print("🔬 Researching...\n")
            print("=" * 50)
            
            stream_obj = client.responses.create(
                model="gpt-4o",
                input=prompt,
                tools=tools,
                text_format=ResearchFindings,
                stream=True
            )
            
            for event in stream_obj:
                if event.type == "response.web_search.searching":
                    print("\n🔍 Searching web...")
                elif event.type == "response.file_search.searching":
                    print("\n📄 Searching documents...")
                elif event.type == "response.output_text.delta":
                    print(event.delta, end="", flush=True)
            
            final = stream_obj.get_final_response()
            print("\n" + "=" * 50)
            return final.output_parsed
        else:
            response = client.responses.parse(
                model="gpt-4o",
                input=prompt,
                tools=tools,
                text_format=ResearchFindings
            )
            return response.output_parsed

### Step 3: Use the Research Agent

In [ ]:
# Create agent
agent = ResearchAgent()

# Conduct research
findings = agent.research(
    "Latest developments in AI agents as of 2025",
    stream=True
)

# Display structured results
print("\n\nRESEARCH FINDINGS")
print("=" * 50)
print(f"Topic: {findings.topic}")
print(f"\nSummary: {findings.summary}")
print(f"\nKey Points:")
for point in findings.key_points:
    print(f"  • {point}")
print(f"\nSources ({len(findings.sources)}):")
for source in findings.sources[:3]:  # Show top 3
    print(f"  [{source.type}] {source.title}")
print(f"\nConfidence: {findings.confidence * 100:.0f}%")

### Production Considerations

When building production agents:

**Error Handling:**
- Handle refusals gracefully
- Retry with exponential backoff
- Validate structured outputs

**Performance:**
- Use streaming for better UX
- Use background mode for long tasks
- Cache results when appropriate

**Monitoring:**
- Log all requests/responses
- Track token usage
- Monitor latency

**Security:**
- Validate all inputs
- Sanitize outputs
- Rate limit API calls
- Never expose API keys

### 🎯 Exercise 7: Build Your Own Agent

**Task:** Extend the research agent with:
1. Comparison mode (compare two topics)
2. Citation formatting
3. Export to markdown
4. Error handling and retries

**Bonus:** Add support for:
- Image analysis
- Background mode for deep research
- Multiple language support

In [ ]:
# YOUR CODE HERE

# Extend ResearchAgent class
# Add new methods and features


---

## Summary

In this notebook, you learned:

✅ **Web Search Tool**: Access real-time information  
✅ **File Search Tool**: RAG with your documents  
✅ **Multimodal Capabilities**: Combine text, images, and more  
✅ **Advanced Streaming**: Event-driven, tool-aware streaming  
✅ **Background Mode**: Long-running tasks with polling  
✅ **Production Agents**: Complete case study with best practices  

**Key Takeaways:**
- Combine multiple tools for powerful agents
- Use streaming for better UX
- Background mode for reasoning models
- Structured outputs ensure reliability
- Handle errors and edge cases
- Monitor performance and costs

**What's Next:**
- Build your own agents!
- Explore LangGraph for more complex workflows
- Deploy to production with proper monitoring

**Resources:**
- [Responses API Documentation](https://platform.openai.com/docs/api-reference/responses)
- [Web Search Guide](https://platform.openai.com/docs/guides/web-search)
- [File Search Guide](https://platform.openai.com/docs/guides/file-search)
- [Streaming Guide](https://platform.openai.com/docs/guides/streaming)
- [Background Mode Guide](https://platform.openai.com/docs/guides/background-mode)